<a href="https://colab.research.google.com/github/AdamsD02/Music-Genre-Classification/blob/main/mgc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd "Music-Genre-Classification"

In [ ]:
%ls

 Install Librosa library for audio analysis and manipulation

In [ ]:
pip install librosa

### Import Libraries

In [33]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Read .csv file containing data for Classifying Genres

In [ ]:
df = pd.read_csv("/content/Music-Genre-Classification/features_30_sec.csv")
# df.head()
# list dataset features
print(df.columns.tolist())

### Manipulate dataframe to:
*   Separate target(X) and features(y)
*   Split into traning and testing (80-20)

In [ ]:
X = df.iloc[:, 2:59] # chroma_stft_mean to mfcc20_var

# Encode non-numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['label'])

print(X.shape)
print(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print("X_train size: ", len(X_train))
print("X_test size: ", len(X_test))

### Use LightGBM as Model Classification and training

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
# clf = lgb.LGBMClassifier(force_col_wise=False)
clf.fit(X_train, y_train)


### Testing and Predictions

In [ ]:
# predict the results
y_pred = clf.predict(X_test)

predicted_genres = le.inverse_transform(y_pred)

mse = np.mean((y_pred - y_test) ** 2)
count = 0
print("Mse: ", mse)
# print(y_pred)
for i in range(len(y_pred)):
  if y_pred[i] - y_test[i] == 0:
    count += 1
  # print("Predicted: ", predicted_genres[i], "\t| Actual: ", le.inverse_transform([y_test[i]]))

print("Accuracy: ", count/len(y_pred))


## Feature Extraction from Uploaded Audio

1.   Load and get sampling rate (sr)
2.   trim audio (y)
3.   sr should match fixed value in dataset, if not then resample
4.   find chroma_stft, then its mean & variance
5.   find rms, then its mean & var
6.   find spectral_centroid, then its mean and var
7.   find spectral_bandwidth, then its mean & var
8.   roll_off, then its mean & var
9.   zero-crossing-rate
10.  harmonics, percetrual
11.  tempo
12.  mfcc with n_mfcc=20


In [55]:
# n_fft=2048, hop_length=512
def get_genre(filepath):
  # Load an MP3 file
  y, sr = librosa.load(filepath, sr=None)  # sr=None keeps original sample rate
  print("Sampling rate:", sr)

  if sr != 22050:
    y = librosa.resample(y=y, orig_sr=sr, target_sr=22050)
    sr = 22050
    print("New Sampling Rate: ", sr)

  features = []

  #get chroma_mean and chroma_var
  chroma = librosa.feature.chroma_stft(y=y, sr=sr)
  chroma_mean = float(np.mean(chroma))
  chroma_var = float(np.var(chroma))
  # print("Chroma Mean:", chroma_mean, "\t | Chroma Variance:", chroma_var)
  features.append(chroma_mean)
  features.append(chroma_var)

  #get rms_mean & rms_var
  rms = librosa.feature.rms(y=y)
  rms_mean = float(np.mean(rms))
  rms_var = float(np.var(rms))
  # print("RMS Mean:", rms_mean, "\t | RMS Variance:", rms_var)
  features.append(rms_mean)
  features.append(rms_var)

  #get spectral-centroid mean & var
  spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  spec_cent_mean = float(np.mean(spec_cent))
  spec_cent_var = float(np.var(spec_cent))
  # print("Spectral Centroid Mean:", spec_cent_mean, "\t | Spectral Centroid Variance:", spec_cent_var)
  features.append(spec_cent_mean)
  features.append(spec_cent_var)

  #get spectral-bandwidth mean & var
  spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
  spec_bw_mean = float(np.mean(spec_bw))
  spec_bw_var = float(np.var(spec_bw))
  # print("Spectral Bandwidth Mean:", spec_bw_mean, "\t | Spectral Bandwidth Variance:", spec_bw_var)
  features.append(spec_bw_mean)
  features.append(spec_bw_var)

  #get roll-off mean & var
  spec_roll_off = librosa.feature.spectral_rolloff(y=y, sr=sr)
  spec_roll_off_mean = float(np.mean(spec_roll_off))
  spec_roll_off_var = float(np.var(spec_roll_off))
  # print("Roll-off Mean:", spec_roll_off_mean, "\t | Roll-off Variance:", spec_roll_off_var)
  features.append(spec_roll_off_mean)
  features.append(spec_roll_off_var)

  #get zero-crossing-rate mean & var
  zcr = librosa.feature.zero_crossing_rate(y)
  zcr_mean = float(np.mean(zcr))
  zcr_var = float(np.var(zcr))
  # print("Zero Crossing Rate Mean:", zcr_mean, "\t | Zero Crossing Rate Variance:", zcr_var)
  features.append(zcr_mean)
  features.append(zcr_var)

  #get harmonics & percetural mean & var
  harm, perc = librosa.effects.hpss(y)
  harm_mean = float(np.mean(harm))
  harm_var = float(np.var(harm))
  perc_mean = float(np.mean(perc))
  perc_var = float(np.var(perc))
  # print("Harmonics Mean:", harm_mean, "\t | Harmonics Variance:", harm_var)
  # print("Percetural Mean:", perc_mean, "\t | Percetural Variance:", perc_var)
  features.append(harm_mean)
  features.append(harm_var)
  features.append(perc_mean)
  features.append(perc_var)

  #get tempo
  tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
  # print("Tempo:", tempo)
  features.append(float(tempo[0]))

  #get mfcc_mean & mfcc_var 1 to 20
  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
  print("MFCC: ", mfcc.shape)
  for i in range(20):
    features.append(float(np.mean(mfcc[i])))
    features.append(float(np.var(mfcc[i])))
    # print("MFCC" + str(i+1) + " Mean:", np.mean(mfcc[i]), "\t | MFCC" + str(i+1) + " Variance:", np.var(mfcc[i]))

  print("Audio shape:", y.shape)
  print("\n Features:\n", features)

  y_pred = clf.predict([features])

  predicted_genres = le.inverse_transform(y_pred)
  print("Predicted Genre: ", predicted_genres)
  return predicted_genres[0]

test get_genre() function using files.upload()

In [ ]:
from google.colab import files

myfile = files.upload()

filename = list(myfile.keys())[0]
print("File uploaded:", filename)

# If needed, get full path
filepath = os.path.join(os.getcwd(), filename)
print("Full path:", filepath)

print(get_genre(filepath))

## GUI with Flask and ngrok

In [ ]:
!pip install flask pyngrok

In [ ]:

import soundfile as sf
from flask import Flask, request, render_template_string
from pyngrok import ngrok, conf
from config import NGROK_TOKEN


app = Flask(__name__)

#set upload directory
UPLOAD_FOLDER = '/content/Music-Genre-Classification/Upload'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# HTML template
form_html = '''
<!DOCTYPE html>
<head>
  <title>Detect Genre</title>
  <style>
    * {
      padding: 0;
      margin: 0;
      box-sizing: border-box;
    }
    body {
      font-family: Consolas, Arial, sans-serif;
      width: 100%;
      height: 99vh;
      background-color: #fff;
      text-align: center;
    }
    h1 {
      font-size: 2.5rem;
      padding-top: 20px;
      margin: auto;
      text-align: center;
      color: #0281f7;
    }
    h2 {
      font-size: 1.5rem;
      margin: auto;
      text-align: center;
      color: #353;
    }
    form {
      margin: 20px auto;
      max-width: 500px;
      padding: 10px;
      text-align: center;
      background-color: #bdf5ff;
      border-radius: 5px;
      box-shadow: 0 1px 8px rgba(0, 0, 0, 0.2);
    }
    input[type="file"] {
      max-width: 80%;
      font-size: 1rem;
      margin: 20px;
    }
    input[type="submit"] {
      margin: 10px, auto;
      width: 250px;
      font-size: 1rem;
      padding: 10px;
      background: #0281f7;
      border: unset;
      border-radius: 5px;
      cursor: pointer;
    }
    section {
      background: #c4ddf5;
      margin: 10px, auto;
      padding: 15px;
      font-size: 1rem;
    }
  </style>
</head>
<body>
  <h1>Detect Genre</h1>
  <h2>Upload an audio file</h2>
  <form method=post enctype=multipart/form-data>
    <input type=file name=audio_file>
    <br>
    <input type=submit value=Upload>
  </form>
  <section>
  {% if genre %}
  <h3>Genre of music is:</h3>
  <pre>{{ genre }}</pre>
  {% elif error %}
  <h3 style="color: #ffaaaa">{{error}}</h3>
  {% endif %}
  </section>
</body>
'''

# Route
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    genre = None
    error = None
    if request.method == 'POST':
        file = request.files['audio_file']
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filepath)
        try:
          # call get_genre()
          genre = get_genre(filepath=filepath)
        except Exception as e:
          error = f"Error processing file: {str(e)}"

    return render_template_string(form_html, genre=genre, error=error)


# ngrok tunnel setup
ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)

#run app
app.run(port=5000)



ngrok's authotoken needs to be saved in a file called config.py